In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import os.path
from bs4 import BeautifulSoup
import requests

In [71]:
record_path_local = '../../../2_data/record_path_mimic3_matched.csv'
record_segment_file_name = '../../../2_data/' + 'record_segment.csv'
record_root_url = 'https://physionet.org/content/mimic3wdb/1.0/matched'
record_path_url = record_root_url + '/RECORDS-waveforms'

signal_extract_name = 'PLETH'
signal_extract_minimum_length = 5 # the minimum length of the signal segment, in minutes

#record_root_url = 'https://archive.physionet.org/physiobank/database/mimic3wdb'
#record_path_url = record_root_url + '/RECORDS'

print('record_path_local: {}'.format(record_path_local))
print('record_root_url: {}'.format(record_root_url))
print('record_path_url: {}'.format(record_path_url))

record_path_local: ../../../2_data/record_path_mimic3_matched.csv
record_root_url: https://physionet.org/content/mimic3wdb/1.0/matched
record_path_url: https://physionet.org/content/mimic3wdb/1.0/matched/RECORDS-waveforms


In [3]:
def get_url_content(url, tag=None):
    
    '''
    This function scrapes a list of useful information from a given PhysioNet URL.
    If the URL address points to an HTML document, the information to be extracted is define by a tag.
    I found this address (https://hackersandslackers.com/scraping-urls-with-beautifulsoup/) usefull
      for details on how to scrape a web page
    '''
    
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }    
    
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    if '<!DOCTYPE html>' in str(soup):
        content = getattr(soup, str(tag)).getText()
        #content = soup.pre.getText()
    else:
        content = soup.getText()
    return content.splitlines()

In [8]:
tag = 'pre'
record_path_list = get_url_content(record_path_url, tag)
print(len(record_path_list))
for i in range(5):
    print('record_path: {}'.format(record_path_list[i]))

22317
record_path: p00/p000020/p000020-2183-04-28-17-47
record_path: p00/p000030/p000030-2172-10-16-12-22
record_path: p00/p000033/p000033-2116-12-24-12-35
record_path: p00/p000033/p000033-2116-12-25-13-11
record_path: p00/p000052/p000052-2191-01-10-02-21


In [74]:
# TO DO:
# we have to decide whether the index_of_last_record_processed is 0 or 1 based 
#  (i.e. it starts from zero or from one)

tag = 'pre'
index_of_last_record_processed = -1
for i in range(index_of_last_record_processed + 1, 20):
        
    #print(i)
    record_path = record_path_list[i]
    print('processing record #: {}, record_path: {}'.format(i, record_path))
    
    #directory p04/p044083.
    directory = str(record_path.split("/")[0]) + '/' + str(record_path.split("/")[1])
    #print('directory: {}'.format(directory))
    
    intermediate_directory = str(record_path.split("/")[0])
    #print('intermediate_directory: {}'.format(intermediate_directory))
    subject_id = record_path.split("/")[1]
    subject_id = int(subject_id[1:])
    #print('subject_id: {}'.format(subject_id))
        
    master_waveform_header = record_root_url + '/' + record_path + '.hea'
    #print('master_waveform_header: {}'.format(master_waveform_header))
    
    target_url = master_waveform_header
    master_waveform_content = get_url_content(target_url, tag)
    #print('master_waveform_content: {}'.format(master_waveform_content))
    
    #print('       ...........\n')
    
    master_waveform_content_first_line = master_waveform_content[0].split(' ')
    record_length = int(master_waveform_content_first_line[3]) / int(master_waveform_content_first_line[2])
    #print('record_length (s): {}'.format(record_length))
    
    #print('       ...........\n')    
        
    waveform_layout_header = master_waveform_content[1].split(' ')[0]
    waveform_layout_header = waveform_layout_header + '.hea'
    waveform_layout_header = record_root_url + '/' + directory + '/' + waveform_layout_header
    #print('waveform_layout_header: {}'.format(waveform_layout_header))    
    target_url = waveform_layout_header
    waveform_layout_content = get_url_content(target_url, tag)
    #print('waveform_layout_content: {}'.format(waveform_layout_content))
        
    signal_names = [x.split(' ')[-1] for x in waveform_layout_content[1:-1]]
    #print('signal_names: {}'.format(signal_names))
    
    if signal_extract_name in signal_names and record_length > 60 * signal_extract_minimum_length: # and len(record_segments_path_list):      
        
        # Open the record_segment_file_name file with access mode 'a'
        record_segment_file_object = open(record_segment_file_name, 'a')

        # get the waveform record name
        waveform_record_name = [i.split(' ')[0].split('_') for i in master_waveform_content if '3' == i[0]][0][0]
        
        # get the waveform_record_state
        waveform_record_state = [(waveform_record_name + '_' in x) and not (waveform_record_name + '_layout' in x) 
                for x in master_waveform_content]

        # get the waveform_record_segment
        waveform_record_segment = [master_waveform_content[i].split(' ') for i, j in enumerate(waveform_record_state) if j]

        
        for i in range(len(waveform_record_segment)):
            # number of samples: int(waveform_record_segment[i][1])
            # sampling frequency: int(master_waveform_content_first_line[2])
            # length (in seconds) of the record segment: number of samples / sampling frequency
            
            if int(waveform_record_segment[i][1]) / int(master_waveform_content_first_line[2]) > 60 * signal_extract_minimum_length:
                
                waveform_record_segment_good_length_root = directory + '/' +  waveform_record_segment[i][0]
                print(waveform_record_segment_good_length_root)
                
                # Append the waveform_record_segment_good_length_root at the end of file record_segment_file_name
                record_segment_file_object.write(waveform_record_segment_good_length_root + '\n')

        print('baubau_1')
        
        # Close the file record_segment_file_name
        record_segment_file_object.close()
        #break
        
    
    
    #print('       ...........\n')
    index_of_last_record_processed = index_of_last_record_processed + 1
    print('  index_of_last_record_processed successfully: {}'.format(index_of_last_record_processed))    
    print('--------------------------------------')

    
    


processing record #: 0, record_path: p00/p000020/p000020-2183-04-28-17-47
  index_of_last_record_processed successfully: 0
--------------------------------------
processing record #: 1, record_path: p00/p000030/p000030-2172-10-16-12-22
  index_of_last_record_processed successfully: 1
--------------------------------------
processing record #: 2, record_path: p00/p000033/p000033-2116-12-24-12-35
  index_of_last_record_processed successfully: 2
--------------------------------------
processing record #: 3, record_path: p00/p000033/p000033-2116-12-25-13-11
  index_of_last_record_processed successfully: 3
--------------------------------------
processing record #: 4, record_path: p00/p000052/p000052-2191-01-10-02-21
  index_of_last_record_processed successfully: 4
--------------------------------------
processing record #: 5, record_path: p00/p000052/p000052-2191-01-10-12-15
  index_of_last_record_processed successfully: 5
--------------------------------------
processing record #: 6, reco